# Tests at the beginnig

In [236]:
import torch
import numpy as np
import matplotlib.pyplot as plt

dataset = []

# -----------------------------Completely unaligned bboxes
pred_bboxes = torch.tensor([
    [1, 1, 1, 1, 1, 1, 0],
    [3, 3, 3, 1, 1, 1, 0],
    [-2, -2, -2, 1, 1, 1, 0]
])
gt_bboxes = torch.tensor([
    [8, 8, 8, 1, 1, 1, 0],
    [-10, -10, -10, 1, 1, 1, 0],
])
dict = {}
dict['pred_bboxes'] = pred_bboxes
dict['gt_bboxes'] = gt_bboxes
dataset.append(dict)



dict1 = dict



# -----------------------------Not enough prediction bboxes (zero)
pred_bboxes = torch.tensor([])
gt_bboxes = torch.tensor([
    [8, 8, 8, 1, 1, 1, 0],
    [-10, -10, -10, 1, 1, 1, 0],
])
dict = {}
dict['pred_bboxes'] = pred_bboxes
dict['gt_bboxes'] = gt_bboxes
dataset.append(dict)



# -----------------------------Not enough prediction bboxes (just one with IoU=1)
pred_bboxes = torch.tensor([
    [8, 8, 8, 1, 1, 1, 0]
])
gt_bboxes = torch.tensor([
    [8, 8, 8, 1, 1, 1, 0],
    [-10, -10, -10, 1, 1, 1, 0],
])
dict = {}
dict['pred_bboxes'] = pred_bboxes
dict['gt_bboxes'] = gt_bboxes
dataset.append(dict)



# -----------------------------Completely aligned bboxes (and extra one)
pred_bboxes = torch.tensor([
    [8, 8, 8, 1, 1, 1, 0],
    [-10, -10, -10, 1, 1, 1, 0],
    [-2, -2, -2, 1, 1, 1, 0]
])
gt_bboxes = torch.tensor([
    [8, 8, 8, 1, 1, 1, 0],
    [-10, -10, -10, 1, 1, 1, 0],
])
dict = {}
dict['pred_bboxes'] = pred_bboxes
dict['gt_bboxes'] = gt_bboxes
dataset.append(dict)



# -----------------------------Completely aligned bboxes
pred_bboxes = torch.tensor([
    [8, 8, 8, 1, 1, 1, 0],
    [-10, -10, -10, 1, 1, 1, 0]
])
gt_bboxes = torch.tensor([
    [8, 8, 8, 1, 1, 1, 0],
    [-10, -10, -10, 1, 1, 1, 0],
])
dict = {}
dict['pred_bboxes'] = pred_bboxes
dict['gt_bboxes'] = gt_bboxes
dataset.append(dict)



# -----------------------------Partially aligned bboxes
pred_bboxes = torch.tensor([
    [8, 8, 8, 1, 1, 1, 0],
    [3, 3, 3, 1, 1, 1, 0],
    [-10, -10, -10, 1, 1, 1, 0]
])
gt_bboxes = torch.tensor([
    [8, 8, 8, 1, 1, 1, 0],
    [-10, -10, -10, 1, 1, 1, 0],
    [-3, 3, 3, 1, 1, 1, 0]
])
dict = {}
dict['pred_bboxes'] = pred_bboxes
dict['gt_bboxes'] = gt_bboxes
dataset.append(dict)



for i in range(10):
    dataset.append(dict1)

In [13]:
def plot_curve(precisions, recalls):
    # Plotting the precision-recall curve
    plt.figure(figsize=(8, 6))
    plt.plot(recalls, precisions, marker='o', linestyle='-', color='b')
    
    # Set x and y axis limits from 0 to 1
    plt.xlim([0, 1.1])
    plt.ylim([0, 1.1])
    
    # Adding labels, title, and grid
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.grid(True)
    
    # Display the plot
    plt.show()

In [238]:
def iou_computer(box1, box2):
    """
    Computes the 3D IoU between two bounding boxes.
    Each box is defined as [x, y, z, dx, dy, dz, yaw].
    For simplicity, we will ignore the yaw for the IoU computation.
    """
    # Extract the dimensions of the boxes
    x1, y1, z1, dx1, dy1, dz1, _ = box1
    x2, y2, z2, dx2, dy2, dz2, _ = box2

    # Calculate the overlap in each dimension
    inter_x = max(0, min(x1 + dx1 / 2, x2 + dx2 / 2) - max(x1 - dx1 / 2, x2 - dx2 / 2))
    inter_y = max(0, min(y1 + dy1 / 2, y2 + dy2 / 2) - max(y1 - dy1 / 2, y2 - dy2 / 2))
    inter_z = max(0, min(z1 + dz1 / 2, z2 + dz2 / 2) - max(z1 - dz1 / 2, z2 - dz2 / 2))

    # Calculate the intersection volume
    inter_volume = inter_x * inter_y * inter_z

    # Calculate the volume of both boxes
    vol1 = dx1 * dy1 * dz1
    vol2 = dx2 * dy2 * dz2

    # Calculate the IoU
    iou = inter_volume / (vol1 + vol2 - inter_volume + 1e-10)
    return iou

In [239]:
def compute_precision_recall(pred_bboxes, gt_bboxes):
    # Initialize the number of true positives and false positives
    tp = 0
    fp = 0
    # Create a set (NO repetition!!) that tracks the indeces of matched ground truths
    matched_gts = set()

    # Many steps:
    #       - cycle through the predictions
    #       - then through the ground truths, to find the best match for the prediction
    #       - when (and if) best match has been found:
    #           - update the tp if over the iou_threshold and not already matched
    #           - otherwise update the fp
    for i in range(pred_bboxes.size(0)):
        # Initialize values
        best_iou = 0
        best_gt_idx = -1
        # Internal for loop
        for j in range(gt_bboxes.size(0)):
            iou = iou_computer(pred_bboxes[i].tolist(), gt_bboxes[j].tolist())
            # Check if the match has to be updated
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = j
        # Update tp or fp
        if best_iou >= 0.5 and best_gt_idx not in matched_gts:
            tp += 1
            matched_gts.add(best_gt_idx)
        else:
            fp += 1

    # False negatives are ground truths that were not matched
    fn = gt_bboxes.size(0) - len(matched_gts)

    # Calculate precision and recall, and return them
    precision = tp / (tp + fp + 1e-15)
    recall = tp / (tp + fn + 1e-15)
    return precision, recall

In [240]:
def compute_ap40(precisions, recalls):
    
    recall_levels = np.linspace(0, 1, 40)
    ap40 = 0.0

    for recall_level in recall_levels:
        # Find the highest precision for the recall level or below
        precisions_at_recall = precisions[recalls >= recall_level]
        if precisions_at_recall.size > 0:
            ap40 += np.max(precisions_at_recall)

    ap40 /= len(recall_levels)
    return ap40

In [ ]:
# Create empty lists for precision and recall
precisions = []
recalls = []


# Cycle through the dictionaries (one for each scan in the dataset)
for dict in dataset:
    # The bboxes come already sorted, so there's no need to sort them again
    pass
    # Compute the precision and recall for the scan
    precision, recall = compute_precision_recall(dict['pred_bboxes'], dict['gt_bboxes'])
    precisions.append(precision)
    recalls.append(recall)
    print("%s \t%s" %(str(precision)[:6], str(recall)[:6]))


# Aggregate all precision-recall curves into a single one
precisions = np.array(sorted(precisions, reverse=True))
recalls = np.array(sorted(recalls))

# Compute AP40
ap40 = compute_ap40(precisions, recalls)


print()
print("-----------------------------------------------------------------------")
print("The metric AP40 is: %s" %ap40)
plot_curve(precisions, recalls)

In [ ]:
recalls = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
precisions = [1.0, 0.96, 0.93, 0.88, 0.85, 0.80, 0.75, 0.7, 0.65, 0.55, 0.5]


# Compute AP40
precisions = np.array(precisions)
recalls = np.array(recalls)
ap40 = compute_ap40(precisions, recalls)


print()
print("-----------------------------------------------------------------------")
print("The metric AP40 is: %s" %ap40)
plot_curve(precisions, recalls)

# plt.scatter(recall)

In [ ]:
tensor1 = torch.Tensor(1, 7)
print(tensor1)

pred_bboxes = torch.tensor([
    [1, 1, 1, 1, 1, 1, 0],
    [3, 3, 3, 1, 1, 1, 0],
    [-2, -2, -2, 1, 1, 1, 0]
])
print(pred_bboxes[0])

tensor1[0] = pred_bboxes[0]
print(tensor1)
print(tensor1.dim())
print(tensor1.size(-1))


# Just plot the curve and compute the AP40

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


#################################################################################################################
#                                           Score_threshold = 0.3                                               #
#################################################################################################################



##################################################       Big Voxels     ##################################################
# precisions = np.array(
#     [0.6097561,  0.53658537, 0.51219512, 0.48780488, 0.48780488, 0.46341463, 0.41463415, 0.41463415, 0.3902439,  0.36585366, 0.34146341, 0.26829268, 0.2195122, 0.2195122, 0.2195122, 0.17073171, 0.17073171, 0.12195122, 0.09756098, 0.09756098, 0.07317073]
#     )
# recalls = np.array(
#     [0.06382979, 0.08510638, 0.08510638, 0.10638298, 0.14893617, 0.14893617, 0.19148936, 0.19148936, 0.19148936, 0.23404255, 0.29787234, 0.31914894, 0.34042553, 0.36170213, 0.36170213, 0.40425532, 0.42553191, 0.42553191, 0.44680851, 0.46808511, 0.53191489]
#     )
# Result: ap40 = 0.1737061280487805



##################################################      Small Voxels    ##################################################
# precisions = np.array(
#     [0.96774194, 0.93548387, 0.93548387, 0.93548387, 0.87096774, 0.83870968, 0.83870968, 0.83870968, 0.80645161, 0.67741935, 0.64516129, 0.5483871, 0.5483871, 0.48387097, 0.35483871, 0.25806452, 0.16129032, 0.09677419, 0.06451613, 0.06451613, 0.03225806]
#     )
# recalls = np.array(
#     [0.0212766, 0.04255319, 0.04255319, 0.06382979, 0.10638298, 0.17021277, 0.23404255, 0.31914894, 0.36170213, 0.36170213, 0.42553191, 0.44680851, 0.53191489, 0.55319149, 0.55319149, 0.55319149, 0.57446809, 0.61702128, 0.61702128, 0.61702128, 0.63829787]
#     )
# Result: ap40 = 0.43273013414634165



##################################################  Reduced_pointcloud  ##################################################
precisions = np.array(
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.97058824, 0.97058824, 0.94117647, 0.88235294, 0.82352941, 0.67647059, 0.55882353, 0.38235294, 0.32352941]
    )
recalls = np.array(
    [0.30555556, 0.36111111, 0.52777778, 0.63888889, 0.77777778, 0.83333333, 0.88888889, 0.91666667, 0.91666667, 0.94444444, 0.94444444, 0.94444444, 0.94444444, 0.94444444, 0.94444444, 0.94444444, 0.94444444, 0.94444444, 0.94444444, 0.94444444, 0.94444444]
    )
# Result: ap40 = 0.926829268292683


def plot_curve(precisions, recalls):
    # Plotting the precision-recall curve
    plt.figure(figsize=(8, 6))
    plt.plot(recalls, precisions, marker='o', linestyle='-', color='b')
    
    # Set x and y axis limits from 0 to 1
    plt.xlim([0, 1.1])
    plt.ylim([0, 1.1])
    
    # Adding labels, title, and grid
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.grid(True)
    
    # Display the plot
    plt.show()

def compute_ap40(precisions, recalls):
    
    recall_levels = np.linspace(0, 1, 41)
    ap40 = 0.0

    for recall_level in recall_levels:
        # Find the highest precision for the recall level or below
        precisions_at_recall = precisions[recalls >= recall_level]
        if precisions_at_recall.size > 0:
            ap40 += np.max(precisions_at_recall)

    ap40 /= len(recall_levels)
    return ap40

plot_curve(precisions, recalls)
print(f"The AP40 value is: {compute_ap40(precisions, recalls)}")
